# Aircraft Safety Analysis for Business Expansion

<img src="images/Flight-Safety-Instructions.jpg">

## Introduction

For my Flatiron School Phase 1 Project, I have been given the following prompt:

Your company is expanding into new industries to diversify its portfolio. Specifically, the company is interested in purchasing and operating airplanes for both commercial and private enterprises. However, the potential risks associated with different aircraft types are unknown. As a data analyst, you have been tasked with determining which aircraft are the lowest risk for the company to start this new business endeavor. Your findings will be translated into actionable insights for the head of the new aviation division to help decide which aircraft to purchase.

## Problem Statement

I will aim to identify which aircraft models present the lowest risk in terms of safety. The key questions I'm seeking to answer include:
- Which aircraft models have the lowest average injury scores?
- How do factors such as the number of engines and the purpose of flight impact the safety of the aircraft?
- Are there temporal trends in the occurrence of accidents or incidents?

## Dataset

The dataset contains records of aircraft incidents and accidents from the NTSB up to 2023, including information on the event, location, aircraft characteristics, and injury severity. The dataset has 90,348 entries and 31 columns. Some key columns include:
- `Event.Id`: Unique identifier for each event.
- `Event.Date`: Date of the event.
- `Location`: Location of the event.
- `Country`: Country where the event occurred.
- `Injury.Severity`: Severity of injuries in the event.
- `Aircraft.damage`: Extent of damage to the aircraft.
- `Make` and `Model`: Manufacturer and model of the aircraft.
- `Number.of.Engines`: Number of engines on the aircraft.
- `Purpose.of.flight`: The purpose of the flight during the event.

## Data Cleaning and Exploration

### Data Cleaning
1. **Handling Missing Values**: Identify and handle missing values appropriately, either by imputation or removal.
2. **Data Type Conversion**: Convert columns to appropriate data types (e.g., dates to datetime objects, numerical columns to numeric types).
3. **Feature Engineering**: Create new features such as `Injury.Severity.Score` to quantify injury severity and extract useful information such as `Event.Year` from the `Event.Date`.

### Exploratory Data Analysis (EDA)
1. **Summary Statistics**: Generate summary statistics for numerical and categorical columns.
2. **Visualizations**: Create histograms, bar plots, and scatter plots to understand the distribution and relationships in the data.
   - **Average Injury Score by Model**: Identify aircraft models with the lowest average injury scores.
   - **Average Injury Score by Number of Engines**: Investigate how the number of engines impacts the injury score.
   - **Average Injury Score by Purpose of Flight**: Explore the impact of flight purpose on safety.
   - **Events by Year**: Visualize temporal trends in the number of incidents.

By following this structured approach, we will be able to gain insights into the safety of different aircraft models and provide actionable recommendations to the company's aviation division.